## Motion Correction using the NoRMCorre algorithm

In [1]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib


import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params


try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl

from movie_visualizer import compare_videos
from utils import start_server

bpl.output_notebook()

c, dview, n_processes = start_server()

Loading BokehJS ...

In [2]:
# Get files to process
fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p1\\19JUN05"
fnames    = [os.path.join(fld, '19JUN05_BF164p1_v1_ds126_crop_ffcSub.tif')]  # ffc filename to be processed

base_name = pathlib.Path(fnames[0]).stem # used to save mmepped data


## Set up MC params

In [6]:
# dataset dependent parameters
frate = 10.                       # movie frame rate
decay_time = 2.              # length of a typical transient in seconds

border_nan = 'copy'

# motion correction parameters
old_mc_dict = { # from vanessa and dom's code
    'gSig_filt': (2,2), # used for high pass filtering in 1p data
    'strides': (30,30), # Can make it shakey
    'overlaps': (11,11), #
    'max_shifts': (25,25), # max shifts in rigid. makes a difference. usually 5-8. can be larger if motion is very rigid
    'max_deviation_rigid': 7, # non-rigid shift deviation from rigid max shifts. makes a difference
    'fnames': fnames, # ! using the ffc subtracted data
    'fr': frate,
    'decay_time': decay_time,
    'border_nan': border_nan,# replicate values along the boundary (if True, fill in with NaN)
    'shifts_opencv': True,
    'num_frames_split':100,
    'nonneg_movie': True,
}
#

mc_dict = { # from here: https://github.com/flatironinstitute/CaImAn/blob/master/demos/general/demo_pipeline_cnmfE.py
    'gSig_filt': (3,3), 
    'strides': (48,48),
    'overlaps': (24,24), 
    'max_shifts': (5,5),
    'max_deviation_rigid': 3,
    'fnames': fnames,
    'fr': frate,
    'decay_time': decay_time,
    'border_nan': border_nan,
    'shifts_opencv': True,
    'num_frames_split':100,
    'nonneg_movie': True,
    'min_mov': 500, #estimated minimum value of the movie to produce an output that is positive; negative vals in movies are bad
}

opts = params.CNMFParams(params_dict=mc_dict)


# Motion Correction
The background signal in micro-endoscopic data is very strong and makes the motion correction challenging. 
As a first step the algorithm performs a high pass spatial filtering with a Gaussian kernel to remove the bulk of the background and enhance spatial landmarks. 
The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no spatial filtering is performed (default option, used in 2p data).
After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data so no information is lost.



## Rigid motion correction

In [7]:
mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

In [8]:
# correct for rigid motion correction and save the file (in memory mapped form)
mc.motion_correct(save_movie=True)


In [9]:
# Now save in C order for CNMF-E 
bord_px = 0 if border_nan is 'copy' else bord_px
fname_new = cm.mmapping.save_memmap(mc.mmap_file, base_name=base_name+"_rig_", order='C', border_to_0=0)
os.remove(mc.mmap_file[0])
print(f"\n\nMotion corrected video was saved at:\n {fname_new}")



Motion corrected video was saved at:
 D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p1\19JUN05\19JUN05_BF164p1_v1_ds126_crop_ffcSub_rig__d1_109_d2_92_d3_1_order_C_frames_22662_.mmap


### Piecewise rigid motion correction

In [ ]:
# motion correct piecewise rigid
mc.pw_rigid = True  # turn the flag to True for pw-rigid motion correction
mc.template = mc.mmap_file  # use the results of the rigid motion corrction to save in computation (optional)
mc.motion_correct(save_movie=True, template=mc.total_template_rig)



In [ ]:
# Now save in C order for CNMF-E 
bord_px = 0 if border_nan is 'copy' else bord_px
fname_new = cm.mmapping.save_memmap(mc.mmap_file, base_name=base_name+"_els_", order='C', border_to_0=bord_px)
os.remove(mc.mmap_file[0])
print(f"\n\nMotion corrected video was saved at:\n {fname_new}")

In [ ]:
### Visualize results

In [ ]:
# load motion corrected movie
m_rig = cm.load(fname_new)

# visualize templates
plt.figure(figsize = (20,10))
plt.imshow(mc.total_template_rig, cmap = 'gray')


# MC quality evaluations
Got it from here here: https://github.com/flatironinstitute/CaImAn/blob/6c5c3b6117b71b6e8b44f62fc26fd3b3d914de12/demos/notebooks/demo_motion_correction.ipynb

!!! THIS DOESN'T CURRENTLY WORK, work in progress...

In [10]:
#% compute metrics for the results (TAKES TIME!!)
final_size = np.subtract(mc.total_template_els.shape, 2 * bord_px_els) # remove pixels in the boundaries
winsize = 100
swap_dim = False
resize_fact_flow = .2    # downsample for computing ROF

tmpl_rig, correlations_orig, flows_orig, norms_orig, crispness_orig = cm.motion_correction.compute_metrics_motion_correction(
    fnames[0], final_size[0], final_size[1], swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)

tmpl_rig, correlations_rig, flows_rig, norms_rig, crispness_rig = cm.motion_correction.compute_metrics_motion_correction(
    mc.fname_tot_rig[0], final_size[0], final_size[1],
    swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)

tmpl_els, correlations_els, flows_els, norms_els, crispness_els = cm.motion_correction.compute_metrics_motion_correction(
    mc.fname_tot_els[0], final_size[0], final_size[1],
    swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)

AttributeError: 'MotionCorrect' object has no attribute 'total_template_els'

In [ ]:
# Plot correlation with mean frame
plt.figure(figsize = (20,10))
plt.subplot(211); plt.plot(correlations_orig); plt.plot(correlations_rig); plt.plot(correlations_els)
plt.legend(['Original','Rigid','PW-Rigid'])
plt.subplot(223); plt.scatter(correlations_orig, correlations_rig); plt.xlabel('Original'); 
plt.ylabel('Rigid'); plt.plot([0.3,0.7],[0.3,0.7],'r--')
axes = plt.gca(); axes.set_xlim([0.3,0.7]); axes.set_ylim([0.3,0.7]); plt.axis('square');
plt.subplot(224); plt.scatter(correlations_rig, correlations_els); plt.xlabel('Rigid'); 
plt.ylabel('PW-Rigid'); plt.plot([0.3,0.7],[0.3,0.7],'r--')
axes = plt.gca(); axes.set_xlim([0.3,0.7]); axes.set_ylim([0.3,0.7]); plt.axis('square');

In [ ]:


# print crispness values
print('Crispness original: '+ str(int(crispness_orig)))
print('Crispness rigid: '+ str(int(crispness_rig)))
print('Crispness elastic: '+ str(int(crispness_els)))



In [ ]:
# plot the results of Residual Optical Flow
fls = [mc.fname_tot_els[0][:-4] + '_metrics.npz', mc.fname_tot_rig[0][:-4] +
       '_metrics.npz', mc.fname[0][:-4] + '_metrics.npz']

plt.figure(figsize = (20,10))
for cnt, fl, metr in zip(range(len(fls)),fls,['pw_rigid','rigid','raw']):
    with np.load(fl) as ld:
        print(ld.keys())
        print(fl)
        print(str(np.mean(ld['norms'])) + '+/-' + str(np.std(ld['norms'])) +
              ' ; ' + str(ld['smoothness']) + ' ; ' + str(ld['smoothness_corr']))
        
        plt.subplot(len(fls), 3, 1 + 3 * cnt)
        plt.ylabel(metr)
        try:
            mean_img = np.mean(
            cm.load(fl[:-12] + 'mmap'), 0)[12:-12, 12:-12]
        except:
            try:
                mean_img = np.mean(
                    cm.load(fl[:-12] + '.tif'), 0)[12:-12, 12:-12]
            except:
                mean_img = np.mean(
                    cm.load(fl[:-12] + 'hdf5'), 0)[12:-12, 12:-12]
                    
        lq, hq = np.nanpercentile(mean_img, [.5, 99.5])
        plt.imshow(mean_img, vmin=lq, vmax=hq)
        plt.title('Mean')
        plt.subplot(len(fls), 3, 3 * cnt + 2)
        plt.imshow(ld['img_corr'], vmin=0, vmax=.35)
        plt.title('Corr image')
        plt.subplot(len(fls), 3, 3 * cnt + 3)
        #plt.plot(ld['norms'])
        #plt.xlabel('frame')
        #plt.ylabel('norm opt flow')
        #plt.subplot(len(fls), 3, 3 * cnt + 3)
        flows = ld['flows']
        plt.imshow(np.mean(
        np.sqrt(flows[:, :, :, 0]**2 + flows[:, :, :, 1]**2), 0), vmin=0, vmax=0.3)
        plt.colorbar()
        plt.title('Mean optical flow')

# Stop cluster

In [ ]:
cm.stop_server(dview=dview)